# Chat API

* Connecting to Claude's API via [claudette](https://claudette.answer.ai/) for chat completion, 
* Extracting JSON and rules to populate gradio UI

In [13]:
#| default_exp chat

In [14]:
#| hide
from nbdev.showdoc import *

In [15]:
#| export
import claudette
from pathlib import Path
import re

* Claudette provides models, which is a list of models currently available from the SDK.

In [109]:
[print(m) for m in claudette.models]
# using Sonnet 3.5
model = claudette.models[2]
print(f'\nChosen model: {model}')

claude-3-opus-20240229
claude-3-7-sonnet-20250219
claude-3-5-sonnet-20241022
claude-3-haiku-20240307
claude-3-5-haiku-20241022

Chosen model: claude-3-5-sonnet-20241022


In [110]:
import os
api_key = os.getenv('ANTHROPIC_API_KEY')
if api_key: print('key exists ☑')

key exists ☑


In [111]:

context_file = Path('docs/analytics.md')
context_media = context_file.read_text(encoding='utf-8')

system_prompt = """You are a helpful and concise assistant."""

chat = claudette.Chat(model, sp=system_prompt)
response = chat("""Can you come up with 10 realistic questions a hiring manager might ask in an opening interview?  the job title for this role is: SENIOR RESEARCH ANALYST,
            Do not add any additional commentary, just list 10 questions only. Start each question with a new line starting like:
                [Q1] ... 
                [Q2] ...
                My resume is in the project docs, the core competencies for a senior analyst are below:
                {context_media}""")


In [112]:
response

[Q1] Can you walk me through your experience with conducting market research and analyzing complex datasets?

[Q2] What statistical analysis tools and software are you most proficient in, and how have you applied them in your previous roles?

[Q3] Describe a challenging research project you led and how you overcame obstacles to deliver actionable insights.

[Q4] How do you ensure accuracy and reliability in your research methodologies and findings?

[Q5] Tell me about a time when you had to present complex data findings to non-technical stakeholders.

[Q6] What strategies do you use to stay current with industry trends and emerging research methodologies?

[Q7] How do you prioritize multiple research projects while maintaining quality and meeting deadlines?

[Q8] Can you describe your experience in mentoring junior analysts and leading research teams?

[Q9] What process do you follow when validating data sources and identifying potential biases in research?

[Q10] Tell me about a time when your research findings led to a significant business decision or strategy change.

<details>

- id: `msg_012hq7WgbkW4R8ZtCPLMdiSc`
- content: `[{'citations': None, 'text': '[Q1] Can you walk me through your experience with conducting market research and analyzing complex datasets?\n\n[Q2] What statistical analysis tools and software are you most proficient in, and how have you applied them in your previous roles?\n\n[Q3] Describe a challenging research project you led and how you overcame obstacles to deliver actionable insights.\n\n[Q4] How do you ensure accuracy and reliability in your research methodologies and findings?\n\n[Q5] Tell me about a time when you had to present complex data findings to non-technical stakeholders.\n\n[Q6] What strategies do you use to stay current with industry trends and emerging research methodologies?\n\n[Q7] How do you prioritize multiple research projects while maintaining quality and meeting deadlines?\n\n[Q8] Can you describe your experience in mentoring junior analysts and leading research teams?\n\n[Q9] What process do you follow when validating data sources and identifying potential biases in research?\n\n[Q10] Tell me about a time when your research findings led to a significant business decision or strategy change.', 'type': 'text'}]`
- model: `claude-3-5-sonnet-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 122, 'output_tokens': 232}`

</details>

## More directly testing on Mock Interviews:

In [113]:
#| export
def get_question_list(qpath):
    with open(qpath, 'r') as file:
        # Read the entire file content as a string
        QUESTION_LIST = file.read()
    return QUESTION_LIST

In [114]:
QUESTION_LIST = get_question_list(qpath="../templates/questions.txt")
print(QUESTION_LIST)

[Q1] Can you walk me through your experience in conducting statistical analysis and predictive modeling to drive product optimization?

[Q2] How have you approached hypothesis testing and A/B experiments in your previous roles? Can you share a specific example?

[Q3] Describe your approach to data quality assurance and how you ensure the integrity of the insights you provide.

[Q4] Can you give an example of a time when you had to communicate complex analytical findings to non-technical stakeholders? How did you tailor your communication style?

[Q5] What is your experience in using data visualization techniques to tell a compelling data story and support business decision-making?

[Q6] How do you stay up-to-date with the latest trends and best practices in the field of data analytics and product research?

[Q7] Can you share a time when you had to navigate ambiguity or uncertainty in your analysis? How did you approach that challenge?

[Q8] Describe your experience in working cross-fu

In [115]:
#| export

INTERVIEW_INSTRUCTIONS = f"""Can you walk through each of the interview question from the file, one by one, and the user 
will reply directly to them in the response, as in a mock interview. Do not engage in small talk or introductions in the
interview part of the exchange. The onversation with the user is to ONLY ask them the questions provided verbatem. 
After their response, I want you to set up the following sections with line breaks between them. 
This metadata will not be immediately shown to the user.

----- ANSWER EVAL ----- Summarise user input answer concisely and into a few bullet points, (depending on length). 
This section will be displayed in Markdown, so format appropriately.
DO NOT be judgemental around users which may come from disadvantaged backgrounds and do not conform to so-called professional
parlance standards, you are here to help everyone from all walks of life.
For EVERY SINGLE RESPONSE, you must take notes sepcified below. They may not seem relevant but they will be logged for 
professional improvement. You can ask a follow up question if the user if completely off-base, but the [Good] [Bad] and 
[Response Metrics] are critically important for our safety and inclusion standards.

Use heading **GOOD**, with a small numbered list of what is considered high-quality response, if if not good, like a joke 
answer or something, you don't need to pretend to be nice. Feel free to use a bit of banter in response. 
Use heading **BAD**, critique of where the answer went wrong, think about verbosity, clarity, what kind of thing should be 
answered in a high level interview, for a very senior position like this. You must be brutally honest. Don't be mean,
be don't flounder around worring about hurt feelings. It's very important this feedback is sharp and accurate. Add some 
humour if you feel it might soften the tone - but it's essential you are concise.]

[RESPONSE METRICS] Here please return a json object with these metrics as keys and a score rating between 0-10 (1 d.p.). 
This is mostly for fun but do the best you can with it. The metrics:
* Linguistic Clarity
* Confidence
* Technical Expertise
* Strategic Thinking
* Impact Demonstration
* Industry Understanding
[ENDJSON]

----- [EDITED_DRAFT] ----- 
Use the above notes to write a first-in-class script for the question. Use the same general facts as the actual answer, 
but don't be afraid to add in external context to the answer if your descretion requires it. This is a fictional exercise 
and we want the answer to be an example of a perfect answer. Use bold to highly key words and phrases in the answer, and 
keep it conversational in tone. It needs to be spoken out loud, so avoid literary flourishes that would only exist on paper. 
It's incredibly important to use your own discretion and not be too verbose! If the question only requires a short answer,
then answer it as succinctly as possibly. If the user gives a short answer, match that kind of vibe, but of course include
additional padding so it conforms to a professionally acceptable length. 
----- [END_DRAFT] ----- 

Once you have fully completed the above instructions, ask the user next interview question in the file.
Continue to do this for each question and answer, and repeat the same process.
Start this header with [Q1], or [Q2] etc.

To remind you, here is the list of questions: {QUESTION_LIST}"""

In [116]:
chat(f"{QUESTION_LIST} {INTERVIEW_INSTRUCTIONS}")

Let's begin with the first question:

[Q1] Can you walk me through your experience in conducting statistical analysis and predictive modeling to drive product optimization?

(Please provide your response, and I'll analyze it according to the specified format.)

<details>

- id: `msg_01CvgbHGffBA8sxhdppCy7AV`
- content: `[{'citations': None, 'text': "Let's begin with the first question:\n\n[Q1] Can you walk me through your experience in conducting statistical analysis and predictive modeling to drive product optimization?\n\n(Please provide your response, and I'll analyze it according to the specified format.)", 'type': 'text'}]`
- model: `claude-3-5-sonnet-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1733, 'output_tokens': 54}`

</details>

In [117]:
user_reply = "Yeah, when I was working as a research analyst, it was pretty tight. Lots of people asked me some pretty hard questions and I, you know, I just crushed it"
c = chat(user_reply)
c

----- ANSWER EVAL -----

**GOOD**
1. Shows confidence in abilities
2. Indicates experience handling challenges

**BAD**
1. Extremely vague and informal response
2. No specific examples provided
3. No mention of statistical analysis or predictive modeling
4. Missing technical details and concrete achievements
5. No demonstration of product optimization experience
6. Too casual for a senior-level interview

[RESPONSE METRICS]
{
    "Linguistic Clarity": 2.0,
    "Confidence": 7.0,
    "Technical Expertise": 1.0,
    "Strategic Thinking": 1.0,
    "Impact Demonstration": 1.0,
    "Industry Understanding": 1.0
}

----- [EDITED_DRAFT] -----
"In my previous role at [Company], I led several key initiatives involving **statistical analysis** and **predictive modeling**. One notable project involved analyzing customer usage patterns to optimize our product features. Using **regression analysis** and **machine learning algorithms**, we identified key drivers of user engagement and developed models that predicted user behavior with 85% accuracy. This led to a **30% increase** in user retention and helped prioritize our product development roadmap. I regularly employed tools like Python, R, and SQL for data manipulation and utilized advanced statistical methods such as **multivariate testing** and **time series analysis** to ensure robust results."

Let's move to the next question:

[Q2] How have you approached hypothesis testing and A/B experiments in your previous roles? Can you share a specific example?

<details>

- id: `msg_01RQ1ebijRsVsv1nejEkRQmz`
- content: `[{'citations': None, 'text': '----- ANSWER EVAL -----\n\n**GOOD**\n1. Shows confidence in abilities\n2. Indicates experience handling challenges\n\n**BAD**\n1. Extremely vague and informal response\n2. No specific examples provided\n3. No mention of statistical analysis or predictive modeling\n4. Missing technical details and concrete achievements\n5. No demonstration of product optimization experience\n6. Too casual for a senior-level interview\n\n[RESPONSE METRICS]\n{\n    "Linguistic Clarity": 2.0,\n    "Confidence": 7.0,\n    "Technical Expertise": 1.0,\n    "Strategic Thinking": 1.0,\n    "Impact Demonstration": 1.0,\n    "Industry Understanding": 1.0\n}\n\n----- [EDITED_DRAFT] -----\n"In my previous role at [Company], I led several key initiatives involving **statistical analysis** and **predictive modeling**. One notable project involved analyzing customer usage patterns to optimize our product features. Using **regression analysis** and **machine learning algorithms**, we identified key drivers of user engagement and developed models that predicted user behavior with 85% accuracy. This led to a **30% increase** in user retention and helped prioritize our product development roadmap. I regularly employed tools like Python, R, and SQL for data manipulation and utilized advanced statistical methods such as **multivariate testing** and **time series analysis** to ensure robust results."\n\nLet\'s move to the next question:\n\n[Q2] How have you approached hypothesis testing and A/B experiments in your previous roles? Can you share a specific example?', 'type': 'text'}]`
- model: `claude-3-5-sonnet-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1827, 'output_tokens': 356}`

</details>

# Test Mechanic – Inline `Gradio UI`

In [118]:
# access claudette messages
c.content[0].text

'----- ANSWER EVAL -----\n\n**GOOD**\n1. Shows confidence in abilities\n2. Indicates experience handling challenges\n\n**BAD**\n1. Extremely vague and informal response\n2. No specific examples provided\n3. No mention of statistical analysis or predictive modeling\n4. Missing technical details and concrete achievements\n5. No demonstration of product optimization experience\n6. Too casual for a senior-level interview\n\n[RESPONSE METRICS]\n{\n    "Linguistic Clarity": 2.0,\n    "Confidence": 7.0,\n    "Technical Expertise": 1.0,\n    "Strategic Thinking": 1.0,\n    "Impact Demonstration": 1.0,\n    "Industry Understanding": 1.0\n}\n\n----- [EDITED_DRAFT] -----\n"In my previous role at [Company], I led several key initiatives involving **statistical analysis** and **predictive modeling**. One notable project involved analyzing customer usage patterns to optimize our product features. Using **regression analysis** and **machine learning algorithms**, we identified key drivers of user eng

In [119]:
claudette.models[2]

'claude-3-5-sonnet-20241022'

In [122]:
import gradio as gr

context_file = Path('docs/analytics.md')
context_media = context_file.read_text(encoding='utf-8')
system_prompt = """You are a helpful and concise assistant."""
model = claudette.models[2]

chat = claudette.Chat(model, sp=system_prompt)
response = chat("""Can you come up with 10 realistic questions a hiring manager might ask in an opening interview?  the job title for this role is: SENIOR RESEARCH ANALYST,
            Do not add any additional commentary, just list 10 questions only. Start each question with a new line starting like:
                [Q1] ... 
                [Q2] ...
                My resume is in the project docs, the core competencies for a senior analyst are below:
                {context_media}""")

def respond(user_message, history):
    print("Messages received so far:", user_message)  # Print to see outside gradio
    c = chat(f'System message: {INTERVIEW_INSTRUCTIONS}, User: {user_message}')
    return c.content[0].text 

chatbot = gr.ChatInterface(
    fn=respond,
    examples=["Hello Neo.. let's begin."],
    theme="shivi/calm_seafoam"
)

chatbot.launch(inline=True) 

d:\ProgramData\Anaconda3.9\envs\wingman\Lib\site-packages\gradio\components\chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


Messages received so far: Hello Neo.. let's begin.
Messages received so far: a lot of other people to do most of the work for me and that product optimization yeah and I take credit for it.


In [90]:
c.content[0].text

'----- ANSWER EVAL -----\n\n**GOOD**\n1. Provided a high-level overview of their experience as a research analyst.\n2. Indicated they were able to handle challenging questions and situations effectively.\n\n**BAD**\nThe response is quite vague and lacks specific details about their background and experience. It would be helpful to hear more concrete examples of the types of research projects they\'ve worked on, the analytical methods they\'ve used, and how they\'ve delivered insights to stakeholders. The language used is also a bit informal for a senior-level interview.\n\n[RESPONSE METRICS]\n{\n  "Linguistic Clarity": 6.0,\n  "Confidence": 7.0,\n  "Technical Expertise": 5.0,\n  "Strategic Thinking": 4.0,\n  "Impact Demonstration": 3.0,\n  "Industry Understanding": 5.0\n}\n\n----- REWRITTEN -----\n\nAs a senior research analyst, I\'ve had the opportunity to work on a wide range of projects that have allowed me to develop a strong set of analytical skills. In my previous role, I was res

In [74]:
#| export
import json 

def extract_metrics(c):
    s1 = (c.content[0].text.split('[RESPONSE METRICS]')[1])
    s2 = s1.split("}")[0] + "}"
    return json.loads(s2)

In [75]:
kpis = extract_metrics(c)
kpis

{'Linguistic Clarity': 5.0,
 'Confidence': 7.0,
 'Technical Expertise': 4.0,
 'Strategic Thinking': 3.0,
 'Impact Demonstration': 3.0,
 'Industry Understanding': 4.0}

In [100]:
def create_dashboard(initial_metrics=None):
    initial_metrics = initial_metrics or {}
    
    with gr.Blocks() as demo:
        # Dict to store all number components
        number_components = {}
        
        # Create all metrics in a single row
        with gr.Row():
            for metric_name, value in initial_metrics.items():
                number_components[metric_name] = gr.Number(
                    label=metric_name,
                    value=value,  # Use the original value without rounding
                    precision=1,  # Set display precision to 1 decimal
                    info="rating/10"
                )
        
        # Function to update metrics
        def update_metrics(new_metrics):
            # Create any new metrics that didn't exist before
            for metric_name in new_metrics:
                if metric_name not in number_components:
                    with gr.Row():
                        number_components[metric_name] = gr.Number(
                            label=metric_name,
                            value=new_metrics[metric_name],
                            precision=2,  # Set display precision
                            info="rating/10"
                        )
            
            # Update all existing metrics
            return [comp.update(value=new_metrics.get(k, 0.0)) for k, comp in number_components.items()]
    
    return demo, update_metrics, number_components

demo, update_metrics, components = create_dashboard(kpis)
demo.launch(inline=True)

* Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.


In [76]:
#| hide
import nbdev; nbdev.nbdev_export()